In [1]:
import pandas as pd

In [4]:
tiker = "ALK"
df = pd.read_csv(f"stocks/data/{tiker}.csv")
print(df.isna().sum())
df.ffill(axis=0, inplace=True)
print(df.isna().sum())

Date                  0
last_traded_price     0
max                  51
min                  51
avg_price             0
promet                0
volume                0
promet_BEST           0
promet_vo_denari      0
dtype: int64
Date                 0
last_traded_price    0
max                  0
min                  0
avg_price            0
promet               0
volume               0
promet_BEST          0
promet_vo_denari     0
dtype: int64


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2679 entries, 0 to 2678
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Date               2679 non-null   object 
 1   last_traded_price  2679 non-null   object 
 2   max                2679 non-null   object 
 3   min                2679 non-null   object 
 4   avg_price          2679 non-null   object 
 5   promet             2679 non-null   object 
 6   volume             2679 non-null   float64
 7   promet_BEST        2679 non-null   object 
 8   promet_vo_denari   2679 non-null   object 
dtypes: float64(1), object(8)
memory usage: 188.5+ KB
